**여기부터**

이탤릭체 텍스트

In [ ]:
import pandas as pd
from google.colab import drive

# ──────────────── 0. Google Drive 연동 ────────────────
drive.mount('/content/drive')

# ──────────────── 1. 공변량 처리 ────────────────
# 공변량 데이터 경로 (파일명과 경로를 정확히 확인할 것!)
merged_df = pd.read_csv('/content/drive/MyDrive/보건의료데이터 공모전/merged_df.csv')

# 필요한 열만 추출
cols = ['ID', 'sex', 'age', 'ho_incm5', 'BD1_11', 'BE5_1']
df = merged_df[cols].copy()

# 나이 필터: 만 12~34세
df = df[(df['age'] >= 12) & (df['age'] <= 34)]

# 음주 빈도 처리
df.loc[df['age'] < 19, 'BD1_11'] = 1  # 미성년자 전부 '전혀 마시지 않음'
df = df[~((df['age'] >= 19) & (df['BD1_11'].isna()))]  # 성인 중 결측 제거

# 근력운동 일수 처리: 비해당/무응답 제거
df = df[~df['BE5_1'].isin([88, 99])]

# 컬럼명 변경
df.rename(columns={
    'sex': '성별',
    'age': '나이',
    'ho_incm5': '가구소득분위',
    'BD1_11': '음주빈도',
    'BE5_1': '주간근력운동일수'
}, inplace=True)

# ID 중복 제거
df = df.drop_duplicates(subset='ID')
print("공변량 데이터 행 개수:", len(df))

# ──────────────── 2. 식품+정신건강 데이터 불러오기 ────────────────
mental_food_df = pd.read_csv('/content/drive/MyDrive/보건의료데이터 공모전/식품+정신건강.csv')
mental_food_df = mental_food_df.drop_duplicates(subset='ID')

# ──────────────── 3. 병합 ────────────────
merged = pd.merge(mental_food_df, df, on='ID', how='inner')
print("최종 병합된 데이터 행 개수:", len(merged))

# 결과 미리 보기
merged.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-376808207.py:9: DtypeWarning: Columns (1,37,38,61,79,114,175,229,237,243,284,319,320,321,324,325,326,329,330,331,332,366,379,381,384,396,398,400,442,590,641,661,673,744,749,828,834,855,857,869,871,873,876,880,990,991,1004,1010,1047,1052,1053,1068,1073,1077,1080,1087,1112,1128,1153,1158,1169,1172,1190,1193,1196,1209,1286,1292) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('/content/drive/MyDrive/보건의료데이터 공모전/merged_df.csv')


공변량 데이터 행 개수: 12278
최종 병합된 데이터 행 개수: 7681


,ID,age,BP1,BP5,BP6_10,BP6_2,BP6_31,BP7,식품코드,식품군 분류,...,당섭취비율_kcal(%),당 에너지 비율 WHO(%),보정 여부,분류,mental_composite,성별,나이_y,가구소득분위,음주빈도,주간근력운동일수
0,YA84242703,12.0,3.0,2.0,2.0,2.0,2.0,2.0,3004.0,당류,...,0.000867,91.697161,X,기타 당류,2.54,2.0,12.0,5.0,1.0,1.0
1,YA84384302,12.0,2.0,2.0,2.0,2.0,2.0,1.0,3014.0,당류,...,0.085726,27.489097,X,기타 당류,2.24,1.0,12.0,3.0,1.0,1.0
2,YA84384303,15.0,3.0,2.0,2.0,2.0,2.0,2.0,8036.0,과일류,...,3.181470,72.600694,X,과일,2.54,1.0,15.0,3.0,1.0,3.0
3,YA84390404,34.0,3.0,2.0,2.0,2.0,2.0,2.0,3014.0,당류,...,0.008047,27.489097,X,기타 당류,2.54,2.0,34.0,5.0,4.0,1.0
4,YA84412603,13.0,3.0,2.0,2.0,2.0,2.0,2.0,13035.0,우유류,...,0.001711,0.322148,X,치즈/고형 유제품,2.54,2.0,13.0,3.0,1.0,6.0


In [ ]:
# ──────────────── 4. 병합 데이터 CSV 저장 ────────────────
save_path = '/content/drive/MyDrive/보건의료데이터 공모전/공변량.csv'
merged.to_csv(save_path, index=False)
print(f"병합된 데이터가 저장되었습니다: {save_path}")


병합된 데이터가 저장되었습니다: /content/drive/MyDrive/보건의료데이터 공모전/공변량.csv


In [ ]:
import pandas as pd

# 파일 경로 설정
base_path = '/content/drive/MyDrive/보건의료데이터 공모전/'

# 1. CSV 파일 불러오기
df_covariate = pd.read_csv(base_path + '공변량.csv')
df_other = pd.read_csv(base_path + 'df00f.csv')

# 2. 병합 전 행 개수 출력
print("공변량.csv 행 개수:", len(df_covariate))
print("df00f.csv 행 개수:", len(df_other))

# 3. ID 기준 병합 (inner join)
merged_df = pd.merge(df_covariate, df_other, on='ID', how='inner')

# 4. 중복 컬럼 제거 (예: suffix 자동 생성된 경우)
# → _x, _y로 구분되는 중복 컬럼 처리
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

# 5. 병합 후 행 개수 출력
print("병합된 데이터 행 개수:", len(merged_df))

# 6. 결과 미리보기 (선택)
merged_df.head()


공변량.csv 행 개수: 7681
df00f.csv 행 개수: 7681
병합된 데이터 행 개수: 7250


,ID,age_x,BP1_x,BP5_x,BP6_10_x,BP6_2_x,BP6_31_x,BP7_x,식품코드_x,식품군 분류_x,...,당섭취열량(kcal)_y,당섭취비율_g(%)_y,당섭취비율_kcal(%)_y,당 에너지 비율 WHO(%)_y,보정 여부_y,분류_y,mental_composite_y,총 당 섭취량(g)_y,주요경로_당_섭취량,주요경로_비율(%)
0,YA84242703,12.0,3.0,2.0,2.0,2.0,2.0,2.0,3004.0,당류,...,0.027917,0.006562,0.000867,91.697161,X,기타 당류,2.54,4573.547496,53.634673,1.172715
1,YA84384302,12.0,2.0,2.0,2.0,2.0,2.0,1.0,3014.0,당류,...,2.947064,0.646698,0.085726,27.489097,X,기타 당류,2.24,683.563686,17.682180,2.586764
2,YA84384303,15.0,3.0,2.0,2.0,2.0,2.0,2.0,8036.0,과일류,...,62.047458,19.319186,3.181470,72.600694,X,과일,2.54,160.585069,15.511864,9.659593
3,YA84390404,34.0,3.0,2.0,2.0,2.0,2.0,2.0,3014.0,당류,...,0.120805,0.041670,0.008047,27.489097,X,기타 당류,2.54,1449.527884,11.058057,0.762873
4,YA84412603,13.0,3.0,2.0,2.0,2.0,2.0,2.0,13035.0,우유류,...,0.031134,0.006433,0.001711,0.322148,X,치즈/고형 유제품,2.54,1209.880532,96.962383,8.014211


In [ ]:
# 병합
merged = pd.merge(df_covariate, df_other, on='ID', how='inner')

# 중복된 컬럼 중에서 _x만 남기고 _y 제거
for col in merged.columns:
    if col.endswith('_y'):
        base_col = col[:-2]
        if f'{base_col}_x' in merged.columns:
            merged.drop(columns=[col], inplace=True)
        else:
            merged.rename(columns={col: base_col}, inplace=True)

# _x 제거
merged.columns = [col[:-2] if col.endswith('_x') else col for col in merged.columns]


In [ ]:
print("공변량 데이터프레임 컬럼:", df_covariate.columns.tolist())
print("df00f 데이터프레임 컬럼:", df_other.columns.tolist())


공변량 데이터프레임 컬럼: ['ID', 'age', 'BP1', 'BP5', 'BP6_10', 'BP6_2', 'BP6_31', 'BP7', '식품코드', '식품군 분류', '식품별 섭취량', '3차 식품코드별 섭취량', '에너지 섭취량', '당 섭취량', '식품코드명', '나이_x', '총 당 섭취량(g)', '총 에너지 섭취량(kcal)', '당섭취열량(kcal)', '당섭취비율_g(%)', '당섭취비율_kcal(%)', '당 에너지 비율 WHO(%)', '보정 여부', '분류', 'mental_composite', '성별', '나이_y', '가구소득분위', '음주빈도', '주간근력운동일수']
df00f 데이터프레임 컬럼: ['ID', 'age', 'BP1', 'BP5', 'BP6_10', 'BP6_2', 'BP6_31', 'BP7', '식품코드', '식품군 분류', '식품별 섭취량', '3차 식품코드별 섭취량', '에너지 섭취량', '당 섭취량', '식품코드명', '총 당 섭취량(g)_x', '총 에너지 섭취량(kcal)', '당섭취열량(kcal)', '당섭취비율_g(%)', '당섭취비율_kcal(%)', '당 에너지 비율 WHO(%)', '보정 여부', '분류', 'mental_composite', '총 당 섭취량(g)_y', '주요경로_당_섭취량', '주요경로_비율(%)']


In [ ]:
print("공변량.csv 행 개수:", len(df_covariate))
print("df00f.csv 행 개수:", len(df_other))
print("병합된 데이터 행 개수:", len(merged))


공변량.csv 행 개수: 7681
df00f.csv 행 개수: 7681
병합된 데이터 행 개수: 7250


In [ ]:
import pandas as pd

# Google Drive 연동
from google.colab import drive
drive.mount('/content/drive')

# 파일 경로
path = '/content/drive/MyDrive/보건의료데이터 공모전/'
file1 = path + '공변량.csv'
file2 = path + 'df00f.csv'

# 데이터 불러오기
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# 병합 (ID 기준 inner join), 중복 열 제거 (_x, _y 제거)
merged_df = pd.merge(df1, df2, on='ID', how='inner')

# 중복된 열 제거 (_x, _y 처리)
for col in merged_df.columns:
    if '_x' in col:
        base_col = col.replace('_x', '')
        if base_col + '_y' in merged_df.columns:
            merged_df[base_col] = merged_df[col]
            merged_df.drop([col, base_col + '_y'], axis=1, inplace=True)

# 병합 결과 출력
print("병합된 데이터:")
display(merged_df)

# 저장
save_path = path + 'df00f_final.csv'
merged_df.to_csv(save_path, index=False)
print(f"저장 완료: {save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
병합된 데이터:


,ID,총 당 섭취량(g),성별,가구소득분위,음주빈도,주간근력운동일수,주요경로_당_섭취량,주요경로_비율(%),age,BP1,...,식품코드명,나이,총 에너지 섭취량(kcal),당섭취열량(kcal),당섭취비율_g(%),당섭취비율_kcal(%),당 에너지 비율 WHO(%),보정 여부,분류,mental_composite
0,YA84242703,106.361570,2.0,5.0,1.0,1.0,53.634673,1.172715,12.0,3.0,...,꿀,12.0,3219.126726,0.027917,0.006562,0.000867,91.697161,X,기타 당류,2.54
1,YA84384302,113.927281,1.0,3.0,1.0,1.0,17.682180,2.586764,12.0,2.0,...,물엿,12.0,3437.751976,2.947064,0.646698,0.085726,27.489097,X,기타 당류,2.24
2,YA84384303,80.292534,1.0,3.0,1.0,3.0,15.511864,9.659593,15.0,3.0,...,"딸기, 개량종, 생것",15.0,1950.276363,62.047458,19.319186,3.181470,72.600694,X,과일,2.54
3,YA84390404,72.476394,2.0,5.0,4.0,1.0,11.058057,0.762873,34.0,3.0,...,물엿,34.0,1501.215719,0.120805,0.041670,0.008047,27.489097,X,기타 당류,2.54
4,YA84412603,120.988053,2.0,3.0,1.0,6.0,96.962383,8.014211,13.0,3.0,...,"치즈, 체다",13.0,1819.705692,0.031134,0.006433,0.001711,0.322148,X,치즈/고형 유제품,2.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7245,R708023903,65.291332,1.0,5.0,1.0,1.0,53.360450,16.345340,14.0,1.0,...,물엿,14.0,2536.095526,0.889882,0.340735,0.035089,27.489097,X,기타 당류,1.94
7246,R708024303,329.147809,1.0,2.0,1.0,1.0,231.815000,7.825427,15.0,3.0,...,"우유,저지방우유(서울우유)",15.0,3607.511516,104.000000,7.899187,2.882874,50.000000,X,우유,2.54
7247,R708024304,113.326030,1.0,2.0,1.0,2.0,43.260000,6.362175,14.0,2.0,...,"우유,저지방우유(서울우유)",14.0,2279.613762,62.400000,13.765593,2.737306,50.000000,X,우유,2.24
7248,R708025102,29.017884,1.0,2.0,1.0,2.0,10.416560,8.974259,13.0,4.0,...,물엿,13.0,1883.987072,0.123913,0.106755,0.006577,27.489097,X,기타 당류,2.84


저장 완료: /content/drive/MyDrive/보건의료데이터 공모전/df00f_final.csv


In [ ]:
import pandas as pd
from google.colab import drive

# ──────────────── 0. Google Drive 연동 ────────────────
drive.mount('/content/drive')

# ──────────────── 1) 공변량 처리 ────────────────
merged_df = pd.read_csv('/content/drive/MyDrive/보건의료데이터 공모전/merged_df.csv')

# 필요 열: + sm_presnt 추가 (BS12_36/46 및 *_clean 열은 사용/보존하지 않음)
cols = ['ID', 'sex', 'age', 'ho_incm5', 'BD1_11', 'BE5_1', 'sm_presnt']
df = merged_df[cols].copy()

print(f"[원본 공변량] 행 수: {len(df)}")

# 나이 필터: 12~34세
df = df[(df['age'] >= 12) & (df['age'] <= 34)]
print(f"[나이 필터] 이후 행 수: {len(df)}")

# 음주 처리
df.loc[df['age'] < 19, 'BD1_11'] = 1          # 미성년자: 전혀 마시지 않음
df = df[~((df['age'] >= 19) & (df['BD1_11'].isna()))]   # 성인 결측 제거
print(f"[음주 처리] 이후 행 수: {len(df)}")

# 근력운동 처리(비해당/무응답 제거)
df = df[~df['BE5_1'].isin([88, 99])]
print(f"[근력운동 처리] 이후 행 수: {len(df)}")

# ──────────────── 2) 흡연여부 생성: 미성년자=비흡연, 성인은 sm_presnt 사용 ────────────────
before_smoke = len(df)

df['흡연여부'] = pd.NA
# 미성년자 전원 비흡연
df.loc[df['age'] < 19, '흡연여부'] = '비흡연'

# 성인: sm_presnt(0=비흡연, 1=현재흡연)로 분류
adult = df['age'] >= 19
df.loc[adult & (df['sm_presnt'] == 1), '흡연여부'] = '흡연'
df.loc[adult & (df['sm_presnt'] == 0), '흡연여부'] = '비흡연'

# 성인에서 sm_presnt 결측이면 제거
df = df[~(adult & df['흡연여부'].isna())]

after_smoke = len(df)
print(f"[흡연 처리] 이전 {before_smoke} → 이후 {after_smoke}")
print(df['흡연여부'].value_counts(dropna=False))

# ──────────────── 3) 컬럼명 정리 ────────────────
df.rename(columns={
    'sex': '성별',
    'age': '나이',
    'ho_incm5': '가구소득분위',
    'BD1_11': '음주빈도',
    'BE5_1': '주간근력운동일수'
}, inplace=True)

# 혹시 남아있을 수 있는 과거 열들 제거
df.drop(columns=['BS12_36', 'BS12_46', 'BS12_36_clean', 'BS12_46_clean'],
        inplace=True, errors='ignore')

# ID 중복 제거
df = df.drop_duplicates(subset='ID')
print(f"[공변량 최종] 중복 제거 후 행 수: {len(df)}")

# ──────────────── 4) 식품+정신건강 병합 ────────────────
mental_food_df = pd.read_csv('/content/drive/MyDrive/보건의료데이터 공모전/식품+정신건강.csv')
mental_food_df = mental_food_df.drop_duplicates(subset='ID')
print(f"[정신건강 데이터] 고유 ID 수: {len(mental_food_df)}")

merged = pd.merge(mental_food_df, df, on='ID', how='inner')
print(f"[최종 병합] 행 수: {len(merged)}")

# 미리보기
merged.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-2906830420.py:8: DtypeWarning: Columns (1,37,38,61,79,114,175,229,237,243,284,319,320,321,324,325,326,329,330,331,332,366,379,381,384,396,398,400,442,590,641,661,673,744,749,828,834,855,857,869,871,873,876,880,990,991,1004,1010,1047,1052,1053,1068,1073,1077,1080,1087,1112,1128,1153,1158,1169,1172,1190,1193,1196,1209,1286,1292) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('/content/drive/MyDrive/보건의료데이터 공모전/merged_df.csv')


[원본 공변량] 행 수: 74952
[나이 필터] 이후 행 수: 16188
[음주 처리] 이후 행 수: 15488
[근력운동 처리] 이후 행 수: 15488
[흡연 처리] 이전 15488 → 이후 15413
흡연여부
비흡연    13219
흡연      2194
Name: count, dtype: int64
[공변량 최종] 중복 제거 후 행 수: 12233
[정신건강 데이터] 고유 ID 수: 7681
[최종 병합] 행 수: 7681


,ID,age,BP1,BP5,BP6_10,BP6_2,BP6_31,BP7,식품코드,식품군 분류,...,보정 여부,분류,mental_composite,성별,나이_y,가구소득분위,음주빈도,주간근력운동일수,sm_presnt,흡연여부
0,YA84242703,12.0,3.0,2.0,2.0,2.0,2.0,2.0,3004.0,당류,...,X,기타 당류,2.54,2.0,12.0,5.0,1.0,1.0,NaN,비흡연
1,YA84384302,12.0,2.0,2.0,2.0,2.0,2.0,1.0,3014.0,당류,...,X,기타 당류,2.24,1.0,12.0,3.0,1.0,1.0,NaN,비흡연
2,YA84384303,15.0,3.0,2.0,2.0,2.0,2.0,2.0,8036.0,과일류,...,X,과일,2.54,1.0,15.0,3.0,1.0,3.0,NaN,비흡연
3,YA84390404,34.0,3.0,2.0,2.0,2.0,2.0,2.0,3014.0,당류,...,X,기타 당류,2.54,2.0,34.0,5.0,4.0,1.0,0.0,비흡연
4,YA84412603,13.0,3.0,2.0,2.0,2.0,2.0,2.0,13035.0,우유류,...,X,치즈/고형 유제품,2.54,2.0,13.0,3.0,1.0,6.0,NaN,비흡연


In [ ]:
import pandas as pd
from google.colab import drive

# ──────────────── 0) Google Drive ────────────────
drive.mount('/content/drive')

# ──────────────── 1) 공변량 로드 & 기본 전처리 ────────────────
merged_df = pd.read_csv('/content/drive/MyDrive/보건의료데이터 공모전/merged_df.csv')

# 사용할 열 (HE_obe 포함)
cols = ['ID', 'sex', 'age', 'ho_incm5', 'BD1_11', 'BE5_1', 'sm_presnt', 'HE_obe']
df = merged_df[cols].copy()

# 혹시 문자열 섞여 있으면 수치화
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['BD1_11'] = pd.to_numeric(df['BD1_11'], errors='coerce')
df['BE5_1'] = pd.to_numeric(df['BE5_1'], errors='coerce')
df['sm_presnt'] = pd.to_numeric(df['sm_presnt'], errors='coerce')
df['HE_obe'] = pd.to_numeric(df['HE_obe'], errors='coerce')

print(f"[원본 공변량] 행 수: {len(df)}")

# 나이 필터: 12~34세
df = df[(df['age'] >= 12) & (df['age'] <= 34)]
print(f"[나이 필터] 이후 행 수: {len(df)}")

# 음주 처리
df.loc[df['age'] < 19, 'BD1_11'] = 1  # 미성년자: 전혀 마시지 않음
df = df[~((df['age'] >= 19) & (df['BD1_11'].isna()))]  # 성인 결측 제거
print(f"[음주 처리] 이후 행 수: {len(df)}")

# 근력운동 처리 (비해당/무응답 제거)
df = df[~df['BE5_1'].isin([88, 99])]
print(f"[근력운동 처리] 이후 행 수: {len(df)}")

# ──────────────── 2) 흡연여부 생성 ────────────────
before_smoke = len(df)

df['흡연여부'] = pd.NA
# 미성년자 전원 비흡연
df.loc[df['age'] < 19, '흡연여부'] = '비흡연'

# 성인: sm_presnt(0=비흡연, 1=현재흡연)
adult_mask = df['age'] >= 19
df.loc[adult_mask & (df['sm_presnt'] == 1), '흡연여부'] = '흡연'
df.loc[adult_mask & (df['sm_presnt'] == 0), '흡연여부'] = '비흡연'

# 성인인데 흡연여부 미정(= sm_presnt 결측)인 행 제거
df = df[~(adult_mask & df['흡연여부'].isna())]

after_smoke = len(df)
print(f"[흡연 처리] 이전 {before_smoke} → 이후 {after_smoke}")
print("[흡연여부 분포]\n", df['흡연여부'].value_counts(dropna=False))

# ──────────────── 3) 비만여부 & 비만단계 생성 (HE_obe 기준) ────────────────
before_obesity = len(df)

# HE_obe 결측 제거
df = df.dropna(subset=['HE_obe'])

# 비만여부(이분화)
df['비만여부'] = df['HE_obe'].apply(lambda x: '비만' if x in [4, 5, 6] else '비비만')

# 비만단계(세분화)
obesity_stage_map = {
    1: '저체중',
    2: '정상',
    3: '비만전단계',
    4: '1단계비만',
    5: '2단계비만',
    6: '3단계비만'
}
df['비만단계'] = df['HE_obe'].map(obesity_stage_map)

after_obesity = len(df)
print(f"[비만 처리] 이전 {before_obesity} → 이후 {after_obesity}")
print("[비만여부 분포]\n", df['비만여부'].value_counts(dropna=False))
print("[비만단계 분포]\n", df['비만단계'].value_counts(dropna=False))

# ──────────────── 4) 컬럼명 정리 ────────────────
df.rename(columns={
    'sex': '성별',
    'age': '나이',
    'ho_incm5': '가구소득분위',
    'BD1_11': '음주빈도',
    'BE5_1': '주간근력운동일수'
}, inplace=True)

# 과거 열 제거(있으면)
df.drop(columns=['BS12_36', 'BS12_46', 'BS12_36_clean', 'BS12_46_clean'],
        inplace=True, errors='ignore')

# ID 중복 제거
df = df.drop_duplicates(subset='ID')
print(f"[공변량 최종] 중복 제거 후 행 수: {len(df)}")

# ──────────────── 5) 식품+정신건강 병합 ────────────────
mental_food_df = pd.read_csv('/content/drive/MyDrive/보건의료데이터 공모전/식품+정신건강.csv')
mental_food_df = mental_food_df.drop_duplicates(subset='ID')
print(f"[정신건강 데이터] 고유 ID 수: {len(mental_food_df)}")

merged = pd.merge(mental_food_df, df, on='ID', how='inner')
print(f"[최종 병합] 행 수: {len(merged)}")

# 결과 확인
merged.head()

# (선택) 저장
# merged.to_csv('/content/drive/MyDrive/보건의료데이터 공모전/merged_final_covariates.csv', index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-1652991886.py:8: DtypeWarning: Columns (1,37,38,61,79,114,175,229,237,243,284,319,320,321,324,325,326,329,330,331,332,366,379,381,384,396,398,400,442,590,641,661,673,744,749,828,834,855,857,869,871,873,876,880,990,991,1004,1010,1047,1052,1053,1068,1073,1077,1080,1087,1112,1128,1153,1158,1169,1172,1190,1193,1196,1209,1286,1292) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('/content/drive/MyDrive/보건의료데이터 공모전/merged_df.csv')


[원본 공변량] 행 수: 74952
[나이 필터] 이후 행 수: 16188
[음주 처리] 이후 행 수: 15488
[근력운동 처리] 이후 행 수: 15488
[흡연 처리] 이전 15488 → 이후 15413
[흡연여부 분포]
 흡연여부
비흡연    13219
흡연      2194
Name: count, dtype: int64
[비만 처리] 이전 15413 → 이후 10244
[비만여부 분포]
 비만여부
비비만    8053
비만     2191
Name: count, dtype: int64
[비만단계 분포]
 비만단계
정상       5154
비만전단계    2010
1단계비만    1587
저체중       889
2단계비만     449
3단계비만     155
Name: count, dtype: int64
[공변량 최종] 중복 제거 후 행 수: 8247
[정신건강 데이터] 고유 ID 수: 7681
[최종 병합] 행 수: 4020


,ID,age,BP1,BP5,BP6_10,BP6_2,BP6_31,BP7,식품코드,식품군 분류,...,성별,나이_y,가구소득분위,음주빈도,주간근력운동일수,sm_presnt,HE_obe,흡연여부,비만여부,비만단계
0,YA84390404,34.0,3.0,2.0,2.0,2.0,2.0,2.0,3014.0,당류,...,2.0,34.0,5.0,4.0,1.0,0.0,4.0,비흡연,비만,1단계비만
1,YA71275102,28.0,3.0,2.0,2.0,2.0,2.0,1.0,95242.0,음료류,...,2.0,28.0,2.0,8.0,1.0,0.0,4.0,비흡연,비만,1단계비만
2,YA71275103,27.0,3.0,2.0,2.0,2.0,2.0,2.0,13010.0,우유류,...,2.0,27.0,2.0,1.0,1.0,0.0,2.0,비흡연,비비만,정상
3,YA71285201,28.0,3.0,1.0,2.0,2.0,2.0,2.0,15087.0,음료류,...,2.0,28.0,2.0,4.0,3.0,0.0,1.0,비흡연,비비만,저체중
4,YA71289202,34.0,3.0,2.0,2.0,2.0,2.0,2.0,13037.0,우유류,...,2.0,34.0,4.0,2.0,4.0,0.0,2.0,비흡연,비비만,정상


In [ ]:
import pandas as pd
from google.colab import drive

# ──────────────── 0. Google Drive 연동 ────────────────
drive.mount('/content/drive')

# ──────────────── 1) 공변량 처리 ────────────────
merged_df = pd.read_csv('/content/drive/MyDrive/보건의료데이터 공모전/merged_df.csv')

# HE_BMI 포함 (HE_obe 제거!) + 흡연 보조변수(BS12_36, BS12_46) 포함
cols = ['ID', 'sex', 'age', 'ho_incm5', 'BD1_11', 'BE5_1',
        'sm_presnt', 'BS12_36', 'BS12_46', 'HE_BMI']
df = merged_df[cols].copy()

print(f"[원본 공변량] 행 수: {len(df)}")

# (선택) 흡연 코드형 캐스팅: 문자로 들어온 경우 대비
for c in ['sm_presnt', 'BS12_36', 'BS12_46', 'HE_BMI']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='ignore')

# 나이 필터: 12~34세
df = df[(df['age'] >= 12) & (df['age'] <= 34)]
print(f"[나이 필터] 이후 행 수: {len(df)}")

# 음주 처리
df.loc[df['age'] < 19, 'BD1_11'] = 1
df = df[~((df['age'] >= 19) & (df['BD1_11'].isna()))]
print(f"[음주 처리] 이후 행 수: {len(df)}")

# 근력운동 처리
df = df[~df['BE5_1'].isin([88, 99])]
print(f"[근력운동 처리] 이후 행 수: {len(df)}")

# ──────────────── 2) 흡연여부 생성 (성인에서 sm_presnt 결측 보완) ────────────────
before_smoke = len(df)

# 미성년자 고정: 비흡연
df['흡연여부'] = pd.NA
df.loc[df['age'] < 19, '흡연여부'] = '비흡연'

adult = df['age'] >= 19

# 2-1) 성인 + sm_presnt 직접 사용 (0=비흡연, 1=현재흡연)
df.loc[adult & (df['sm_presnt'] == 1), '흡연여부'] = '흡연'
df.loc[adult & (df['sm_presnt'] == 0), '흡연여부'] = '비흡연'

# 2-2) 성인 + sm_presnt가 NaN → BS12_36로 보완
# BS12_36: 평생 5갑 이상(1=예 → 흡연), 2=아니오, 8=비해당 → 비흡연, 9=모름(보완 불가)
missing_smoke = adult & df['흡연여부'].isna()
df.loc[missing_smoke & (df['BS12_36'] == 1), '흡연여부'] = '흡연'
df.loc[missing_smoke & (df['BS12_36'].isin([2, 8])), '흡연여부'] = '비흡연'

# 2-3) 여전히 NaN → BS12_46로 보완
# BS12_46: 현재 흡연(1=매일,2=가끔 → 흡연), 3=전혀 → 비흡연, 8/9=모름/비해당(보완 불가)
missing_smoke = adult & df['흡연여부'].isna()
df.loc[missing_smoke & (df['BS12_46'].isin([1, 2])), '흡연여부'] = '흡연'
df.loc[missing_smoke & (df['BS12_46'] == 3), '흡연여부'] = '비흡연'

# 2-4) 성인인데도 여전히 미결정(na) → 제거
df = df[~(adult & df['흡연여부'].isna())]

after_smoke = len(df)
print(f"[흡연 처리(보완 포함)] 이전 {before_smoke} → 이후 {after_smoke}")
print(df['흡연여부'].value_counts(dropna=False))

# ──────────────── 3) 비만여부 및 단계 생성 (HE_BMI 기준) ────────────────
before_bmi = len(df)
df = df[~df['HE_BMI'].isna()]  # 결측 제거

# 비만여부 (대한비만학회 기준: 25 이상 비만)
df['비만여부'] = df['HE_BMI'].apply(lambda x: '비만' if x >= 25 else '비비만')

# 비만단계
def classify_bmi(bmi):
    if bmi < 18.5:
        return '저체중'
    elif bmi < 23:
        return '정상체중'
    elif bmi < 25:
        return '과체중'
    elif bmi < 30:
        return '1단계 비만'
    else:
        return '2단계 이상 비만'

df['비만단계'] = df['HE_BMI'].apply(classify_bmi)

after_bmi = len(df)
print(f"[BMI 처리] 이전 {before_bmi} → 이후 {after_bmi}")
print(df['비만단계'].value_counts(dropna=False))

# ──────────────── 4) 컬럼 정리 ────────────────
df.rename(columns={
    'sex': '성별',
    'age': '나이',
    'ho_incm5': '가구소득분위',
    'BD1_11': '음주빈도',
    'BE5_1': '주간근력운동일수'
}, inplace=True)

# 흡연 원시 변수 및 HE_obe 제거
df.drop(columns=['sm_presnt', 'BS12_36', 'BS12_46', 'HE_obe', 'BS12_36_clean', 'BS12_46_clean'],
        inplace=True, errors='ignore')

df = df.drop_duplicates(subset='ID')
print(f"[공변량 최종] 중복 제거 후 행 수: {len(df)}")

# ──────────────── 5) 식품+정신건강 병합 ────────────────
mental_food_df = pd.read_csv('/content/drive/MyDrive/보건의료데이터 공모전/식품+정신건강.csv')
mental_food_df = mental_food_df.drop_duplicates(subset='ID')
print(f"[정신건강 데이터] 고유 ID 수: {len(mental_food_df)}")

merged = pd.merge(mental_food_df, df, on='ID', how='inner')
print(f"[최종 병합] 행 수: {len(merged)}")

# 미리보기
merged.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-3833615090.py:8: DtypeWarning: Columns (1,37,38,61,79,114,175,229,237,243,284,319,320,321,324,325,326,329,330,331,332,366,379,381,384,396,398,400,442,590,641,661,673,744,749,828,834,855,857,869,871,873,876,880,990,991,1004,1010,1047,1052,1053,1068,1073,1077,1080,1087,1112,1128,1153,1158,1169,1172,1190,1193,1196,1209,1286,1292) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('/content/drive/MyDrive/보건의료데이터 공모전/merged_df.csv')


[원본 공변량] 행 수: 74952
[나이 필터] 이후 행 수: 16188
[음주 처리] 이후 행 수: 15488
[근력운동 처리] 이후 행 수: 15488
[흡연 처리(보완 포함)] 이전 15488 → 이후 15413
흡연여부
비흡연    13219
흡연      2194
Name: count, dtype: int64
[BMI 처리] 이전 15413 → 이후 15009
비만단계
정상체중         6873
1단계 비만       2882
과체중          2329
저체중          1950
2단계 이상 비만     975
Name: count, dtype: int64
[공변량 최종] 중복 제거 후 행 수: 11933


/tmp/ipython-input-3833615090.py:20: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors='ignore')


[정신건강 데이터] 고유 ID 수: 7681
[최종 병합] 행 수: 7635


,ID,age,BP1,BP5,BP6_10,BP6_2,BP6_31,BP7,식품코드,식품군 분류,...,mental_composite,성별,나이_y,가구소득분위,음주빈도,주간근력운동일수,HE_BMI,흡연여부,비만여부,비만단계
0,YA84242703,12.0,3.0,2.0,2.0,2.0,2.0,2.0,3004.0,당류,...,2.54,2.0,12.0,5.0,1.0,1.0,17.641364,비흡연,비비만,저체중
1,YA84384302,12.0,2.0,2.0,2.0,2.0,2.0,1.0,3014.0,당류,...,2.24,1.0,12.0,3.0,1.0,1.0,24.508946,비흡연,비비만,과체중
2,YA84384303,15.0,3.0,2.0,2.0,2.0,2.0,2.0,8036.0,과일류,...,2.54,1.0,15.0,3.0,1.0,3.0,27.837219,비흡연,비만,1단계 비만
3,YA84390404,34.0,3.0,2.0,2.0,2.0,2.0,2.0,3014.0,당류,...,2.54,2.0,34.0,5.0,4.0,1.0,25.807581,비흡연,비만,1단계 비만
4,YA84412603,13.0,3.0,2.0,2.0,2.0,2.0,2.0,13035.0,우유류,...,2.54,2.0,13.0,3.0,1.0,6.0,20.931313,비흡연,비비만,정상체중


In [ ]:
# ──────────────── 6) 최종 데이터 저장 ────────────────
save_path = '/content/drive/MyDrive/보건의료데이터 공모전/공변량_최종.csv'
merged.to_csv(save_path, index=False, encoding='utf-8-sig')
print(f"[저장 완료] 경로: {save_path}")


[저장 완료] 경로: /content/drive/MyDrive/보건의료데이터 공모전/공변량_최종.csv


In [ ]:
import pandas as pd

# 파일 경로 설정
path1 = '/content/drive/MyDrive/보건의료데이터 공모전/공변량_최종.csv'
path2 = '/content/drive/MyDrive/보건의료데이터 공모전/df00f_final.csv'

# 데이터 불러오기
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

# 중복 컬럼 탐색 (ID 제외)
common_cols = [col for col in df1.columns if col in df2.columns and col != 'ID']

# df2에서 중복 컬럼 제거
df2_deduped = df2.drop(columns=common_cols)

# 병합
merged_df = pd.merge(df1, df2_deduped, on='ID', how='inner')

# 저장
save_path = '/content/drive/MyDrive/보건의료데이터 공모전/병합_최종.csv'
merged_df.to_csv(save_path, index=False, encoding='utf-8-sig')
print(f"[병합 및 저장 완료] 경로: {save_path}")


[병합 및 저장 완료] 경로: /content/drive/MyDrive/보건의료데이터 공모전/병합_최종.csv


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

file_path = '/content/drive/MyDrive/보건의료데이터 공모전/final.csv'
df = pd.read_csv(file_path)
# 각 컬럼별로 결측값이 아닌 응답 개수 출력
df.count()
response_count = df.count()
total = len(df)
response_rate = (response_count / total * 100).round(2)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
